# Azure Machine Learning Responsible AI MLOps

## Overview

Read [Azure Machine Learning Pipelines](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-ml-pipelines) overview, or the [readme article](../README.md) on Azure Machine Learning Pipelines to get more information.
 

This notebook shows the construction of a machine learning service **pipeline** with Responsible AI steps and MLOps techniques that runs jobs unattended in different compute clusters.

## Prerequisites and Azure Machine Learning Basics

Please, before anything set up with a working config file that has information on your workspace, subscription id, etc located on:

- './notebooks/notebook-settings/config.json' 

## Imports

In [ ]:
!pip install numpy==1.18.5
!pip install pandas==1.0.4
!pip install ipykernel==5.1.4
!pip install jupyter==1.0.0
!pip install sklearn==0.0
!pip install scikit-learn==0.23.0
!pip install matplotlib==3.1.0
!pip install interpret-community==0.11.2
!pip install fairlearn==0.4.6
!pip install azureml-core==1.13.0
!pip install azureml-pipeline-core==1.13.0
!pip install azureml-sdk==1.8.0
!pip install azureml-sdk[automl]
!pip install azureml-widgets==1.8.0
!pip install azureml-contrib-fairness==1.8.0
!pip install azureml-defaults>=1.0.45
!pip install azureml-sdk[automl]==1.7.0.post1
!pip install azureml-pipeline-steps
!pip install azureml-interpret==1.7.0.post1
!pip install azureml-contrib-interpret==1.7.0
!pip install azureml-monitoring==0.1.0a18
!pip install opendp-whitenoise==0.1.1.1
!pip install opendp-whitenoise-core==0.1.3
!pip install joblib==0.16.0
!pip install azureml-datadrift==1.8.0
!pip install seaborn==0.10.0
!pip install pandas-profiling==2.8.0

In [ ]:
import os
import sys
import requests
import azureml.core
import warnings
warnings.filterwarnings('ignore')

from azureml.widgets import RunDetails
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget
from azureml.pipeline.core import PipelineEndpoint
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.core import Workspace, Run, Experiment, Datastore, Dataset
from azureml.core.runconfig import RunConfiguration, CondaDependencies
from azureml.core.authentication import InteractiveLoginAuthentication

sys.path.append(os.path.abspath("../utils"))
from attach_compute import get_compute_aml
from workspace import get_workspace
from dataset import upload_dataset,convert_dataset_columns
from pipeline_parameter_builder import PipelineParameterBuilder

## Pipeline-specific SDK imports

Here, we import key pipeline modules, whose use will be illustrated in the examples below.

In [ ]:
from azureml.data import DataType
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData, StepSequence, TrainingOutput
from azureml.pipeline.steps import PythonScriptStep, AutoMLStep
from azureml.pipeline.core import PublishedPipeline, PipelineRun
from azureml.pipeline.core.graph import PipelineParameter

print("Pipeline SDK-specific imports completed")

### Initialize Workspace

Now you're ready to connect to your workspace using the Azure ML SDK.

Note: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [ ]:
ws = Workspace.from_config("../notebooks-settings/config.json")
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Get the Default datastore (Azure Blob storage)

### Datastore concepts
A [Datastore](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.datastore(class) is a place where data can be stored that is then made accessible to a compute either by means of mounting or copying the data to the compute target. 

A Datastore can either be backed by an Azure File Storage (default) or by an Azure Blob Storage.

In this next step, we will upload the training and test set into the workspace's default storage (File storage), and another piece of data to Azure Blob Storage. When to use [Azure Blobs](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-blobs-introduction), [Azure Files](https://docs.microsoft.com/en-us/azure/storage/files/storage-files-introduction), or [Azure Disks](https://docs.microsoft.com/en-us/azure/virtual-machines/linux/managed-disks-overview) is [detailed here](https://docs.microsoft.com/en-us/azure/storage/common/storage-decide-blobs-files-disks).

**Please take good note of the concept of the datastore.**

Now we will get Blob storage associated with the workspace.

The following call GETS the Azure Blob Store associated with your workspace.

Note that workspaceblobstore is **the name of this store and CANNOT BE CHANGED and must be used as is**

The above call is equivalent to Datastore(ws, "workspaceblobstore") or simply Datastore(ws)

In [ ]:
def_blob_store = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_store.name))

## Required data and script files for the the tutorial
Sample files required to finish this tutorial are already copied to the project folder specified above. Even though the .py provided in the samples don't have much "ML work," as a data scientist, you will work on this extensively as part of your work. To complete this tutorial, the contents of these files are not very important. The one-line files are for demostration purpose only.

#### Upload data to default datastore
Default datastore on workspace is the Azure  File storage. The workspace has a Blob storage associated with it as well. Let's upload a file to each of these storages. 

get_default_datastore() gets the default Azure File Store associated with your workspace. Here we are reusing the def_file_store object we obtained earlier

Parameters of upload_dataset:

1. Workspace object
2. Blob Storage Datastore object
3. Azure Dataset name
4. Local path on Datastore
5. Local path of dataset
6. This dataset will be use on Datadrift Detector (True/False)
7. dataset type. The original model (uci_dataset.csv and preprocessed dataset will be standard). Just dataset with type "train" and inference will use on Datadrift

In the next cell we'll upload:

1. The complete dataset about heart-disease patients. This dataset contain information about heart disease relevent factors to predict if the patient need a treatment or not but also this initial dataset have sensible information such as State, City, Address, Observation... with this information we could identify each row on the dataset.

In [ ]:
upload_dataset(ws, def_blob_store, 'heart_disease_dataset',
                                  'heart-disease/complete_patients_dataset.csv', "../../dataset/complete_patients_dataset.csv",
                                  use_datadrift=False, type_dataset="Standard")

#### (Optional) See your files using Azure Portal
Once you successfully uploaded the files, you can browse to them (or upload more files) using [Azure Portal](https://portal.azure.com). At the portal, make sure you have selected your subscription (click *Resource Groups* and then select the subscription). Then look for your **Machine Learning Workspace** (it has your *alias* as the name). It has a link to your storage. Click on the storage link. It will take you to a page where you can see [Blobs](https://docs.microsoft.com/en-us/azure/storage/blobs/storage-blobs-introduction), [Files](https://docs.microsoft.com/en-us/azure/storage/files/storage-files-introduction), [Tables](https://docs.microsoft.com/en-us/azure/storage/tables/table-storage-overview), and [Queues](https://docs.microsoft.com/en-us/azure/storage/queues/storage-queues-introduction). We have just uploaded a file to the Blob storage and another one to the File storage. You should be able to see both of these files in their respective locations. 

## Compute Targets
A compute target specifies where to execute your program such as a remote Docker on a VM, or a cluster. A compute target needs to be addressable and accessible by you.

**You need at least one compute target to send your payload to. We are planning to use Azure Machine Learning Compute exclusively for this tutorial for all steps. However in some cases you may require multiple compute targets as some steps may run in one compute target like Azure Machine Learning Compute, and some other steps in the same pipeline could run in a different compute target.**

*The example belows show creating/retrieving/attaching to an Azure Machine Learning Compute instance.*

#### List of Compute Targets on the workspace

In [ ]:
cts = ws.compute_targets
for ct in cts:
    print(ct)

#### Retrieve or create a Azure Machine Learning compute
Azure Machine Learning Compute is a service for provisioning and managing clusters of Azure virtual machines for running machine learning workloads. Let's create a new Azure Machine Learning Compute in the current workspace, if it doesn't already exist. We will then run the training script on this compute target.

If we could not find the compute with the given name in the previous cell, then we will create a new compute here. We will create an Azure Machine Learning Compute containing **STANDARD_D2_V2 CPU VMs**. This process is broken down into the following steps:

1. Create the configuration
2. Create the Azure Machine Learning compute

**This process will take about 3 minutes and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell.**

In [ ]:
aml_compute_name = "aml-compute"
vm_size = "STANDARD_DS3_V2"
aml_compute = get_compute_aml(ws, aml_compute_name, vm_size)
print("Azure Machine Learning Compute attached")

## Create MLOps Pipeline Compute

Pipelines consist of one or more steps, which can be Python scripts, or specialized steps like an Auto ML training estimator or a data transfer step that copies data from one location to another. Each step can run in its own compute context.

In this notebook, you'll build a Responsible AI pipeline that contains nine steps, which include the following steps:

1. **Differential Privacy**
2. **Exploratory Analysis and Preprocessing**
3. **AutoML Training/Evaluation**
4. **Detect AutoML Fairness** 
5. **Upload AutoML Model Fairness**
6. **Build Fairlearn Multi Model View**
7. **Register AutoML Model**
8. **Register Explainer Model**
9. **Deploy AutoML and Explainer Model**

## Create pipeline steps run configs

In this notebook, you'll use the same compute for both steps, but it's important to realize that each step is run independently; so you could specify different compute contexts for each step if appropriate.

First, get the compute target you created in a previous cells (if it doesn't exist, it will be created).
Each step of the pipeline will need some dependencies to execute and generate well results. So, because of this need, we break down the run configurations and focus on each step and see what dependencies each needs.

#### Run Config - Analysis and preprocessing step

In [ ]:
run_preprocessing_config = RunConfiguration(conda_dependencies=CondaDependencies.create(
        conda_packages=['numpy==1.18.1', 'pandas==1.0.4'],
        pip_packages=['azureml-sdk', 'matplotlib==3.1.3', 'seaborn==0.10.0', 'sklearn==0.0', 'pandas-profiling==2.8.0'])
    )
run_preprocessing_config.environment.docker.enabled = True

#### Run Config - AutoML step

In [ ]:
run_automl_config = RunConfiguration(conda_dependencies=CondaDependencies.create(
        conda_packages=[],
        pip_packages=['azureml-sdk', 'azureml-sdk[automl]', 'azureml-train-automl-runtime==1.13.0', 'xgboost==1.1.1'])
    )
run_automl_config.environment.docker.enabled = True

#### Run Config - Fairlearn Detect Fairness step

In [ ]:
run_fairlearn_detect_fairness_config = RunConfiguration(conda_dependencies=CondaDependencies.create(
        conda_packages=[],
        pip_packages=['azureml-sdk', 'interpret-community==0.11.2',
                      'azureml-train-automl-runtime==1.8.0.post1', 'fairlearn==0.4.6',
                      'azureml-contrib-fairness==1.8.0', 'matplotlib==3.1.3', 'scikit-learn==0.20.3', 'xgboost==1.1.1'])
    )
run_fairlearn_detect_fairness_config.environment.docker.enabled = True

#### Run Config - Fairlearn Create Multimodels Fairness step

In [ ]:
run_fairlearn_create_multimodel_view_config = RunConfiguration(conda_dependencies=CondaDependencies.create(
        conda_packages=[],
        pip_packages=['azureml-sdk', 'interpret-community==0.11.2', 'fairlearn==0.4.6',
                      'azureml-contrib-fairness==1.8.0', 'matplotlib==3.1.3', 'scikit-learn==0.23.0', 'xgboost==1.1.1'])
    )
run_fairlearn_create_multimodel_view_config.environment.docker.enabled = True

#### Run Config - Explainer Model step

In [ ]:
run_explainer_model_config = RunConfiguration(conda_dependencies=CondaDependencies.create(
        conda_packages=[],
        pip_packages=['azureml-sdk', 'azureml-sdk[explain]', 'azureml-train-automl-runtime==1.13.0', 'scikit-learn==0.21.0','xgboost==1.1.1'])
    )
run_explainer_model_config.environment.docker.enabled = True

#### Run Config - Differential Privacy step

In [ ]:
run_differential_privacy_config = RunConfiguration(conda_dependencies=CondaDependencies.create(
        conda_packages=['pandas==1.0.4'],
        pip_packages=['azureml-sdk', 'opendp-whitenoise==0.1.1.1', 'opendp-whitenoise-core==0.1.3', 'matplotlib==3.1.3'])
    )
run_differential_privacy_config.environment.docker.enabled = True

**Now that we have completed learning the basics of Azure Machine Learning (AML), let's go ahead and start understanding the Pipeline concepts.**

## Creating Steps in a Pipeline
A Step is a unit of execution. Step typically needs a target of execution (compute target), a script to execute, and may require script arguments and inputs, and can produce outputs. The step also could take a number of other parameters. Azure Machine Learning Pipelines provides the following built-in Steps:

- [**PythonScriptStep**](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.python_script_step.pythonscriptstep?view=azure-ml-py): Add a step to run a Python script in a Pipeline.

- [**AutoML Step**](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.automl_step.automlsteprun?view=azure-ml-py): Creates a AutoML step to manage, check status, and retrieve run details once an automated ML run is submitted in a pipeline.

- [**AdlaStep**](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.adla_step.adlastep?view=azure-ml-py): Adds a step to run U-SQL script using Azure Data Lake Analytics.

- [**DataTransferStep**](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.data_transfer_step.datatransferstep?view=azure-ml-py): Transfers data between Azure Blob and Data Lake accounts.

- [**DatabricksStep**](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.databricks_step.databricksstep?view=azure-ml-py): Adds a DataBricks notebook as a step in a Pipeline.

- [**HyperDriveStep**](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.hyper_drive_step.hyperdrivestep?view=azure-ml-py): Creates a Hyper Drive step for Hyper Parameter Tuning in a Pipeline.

The following code will create a PythonScriptStep to be executed in the Azure Machine Learning Compute we created above using train.py, one of the files already made available in the project folder.

A **PythonScriptStep** is a basic, built-in step to run a Python Script on a compute target. It takes a script name and optionally other parameters like arguments for the script, compute target, inputs and outputs. If no compute target is specified, default compute target for the workspace is used.

## Configure PipelineData

Here, we configure some outputs on the pipeline. The first output (preprocess_step_output) will be between **Exploratory Analysis and Preprocessing** and **AutoML Training/Evaluation**. Second one (fairness_predictions), will be between Detect Fairness and Upload AutoML Fairness Insights step.

In [ ]:
preprocess_step_output = PipelineData("hd_preprocessed", datastore=def_blob_store).as_dataset()

In [ ]:
detect_fairness_step_output = PipelineData("fairness_predictions", datastore=def_blob_store)

### Convert Dataset Columns to PipelineDataType

In [ ]:
data_dict = convert_dataset_columns(ws,'./scripts/schema_dataset.json')

## Pipeline Parameters Builder

In [ ]:
ppb = PipelineParameterBuilder("../utils/params_config/pipeline_parameters.json")
ppb.build()

## Configure AutoML 

In [ ]:
automl_settings = {
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted',
    "featurization": 'auto',
    'model_explainability': True,
    'iterations': 10,
    'validation_size': 0.2,
    'enable_early_stopping': True,
    'label_column_name': 'target',
    'task': 'classification'
}
automl_config = AutoMLConfig(compute_target=aml_compute,
                             training_data = preprocess_step_output.parse_delimited_files(set_column_types=data_dict),
                             path = ".",
                             **automl_settings
                            )

In [ ]:
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=def_blob_store,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=def_blob_store,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

## Running Pipeline

Now you're ready to create and run a pipeline. First you need to define the steps for the pipeline, and any data references that need to passed between them. The Pipeline Data and Parameters configurations were created in the last cells.

In [ ]:
project_folder = './scripts'

In [ ]:
deploy_folder = '../deployment'

In [ ]:
differential_privacy_step = PythonScriptStep(name="Differential Privacy",
                         script_name="differential_privacy.py", 
                         compute_target=aml_compute,
                         source_directory=project_folder,
                         arguments=[
                              "--datastore", ppb.get_pipeline_parameter_obj('datastore_differential_privacy_step'),
                              "--dataset_name", ppb.get_pipeline_parameter_obj('dataset_name_differential_privacy_step'),
                              "--retrain_status", ppb.get_pipeline_parameter_obj('retrain_status_differential_privacy_step')
                         ],
                         runconfig=run_differential_privacy_config,
                         allow_reuse=False)

preprocessing_step = PythonScriptStep(name="Exploratory Analysis and Preprocessing",
                         script_name="preprocessing.py",
                         compute_target=aml_compute, 
                         source_directory=project_folder,
                         arguments=[
                              "--datastore", ppb.get_pipeline_parameter_obj('datastore_preprocessing_step'),
                              "--dataset_name", ppb.get_pipeline_parameter_obj('dataset_name_preprocessing_step'),
                              "--dataset_preprocessed_name", ppb.get_pipeline_parameter_obj('dataset_preprocessed_name_preprocessing_step'),
                              "--output_preprocess_dataset", preprocess_step_output,
                              "--use_datadrift", ppb.get_pipeline_parameter_obj('use_datadrift_name_preprocessing_step'),
                              "--retrain_status", ppb.get_pipeline_parameter_obj('retrain_status_preprocessing_step')
                         ],
                         outputs=[preprocess_step_output],
                         runconfig=run_preprocessing_config,
                         allow_reuse=False)

automl_step = AutoMLStep(
                name='AutoML_training',
                automl_config=automl_config,
                inputs = [preprocess_step_output],
                outputs=[metrics_data, model_data],
                passthru_automl_config=False,
                allow_reuse=False)

register_model_step = PythonScriptStep(script_name="register_model.py",
                                  source_directory=project_folder,
                                  name="Register Model",
                                  arguments=["--model_name", ppb.get_pipeline_parameter_obj('model_name_register_model_step'),
                                             "--model_data", model_data,
                                             "--metrics_data", metrics_data,
                                             "--dataset_name", ppb.get_pipeline_parameter_obj('dataset_name_register_model_step')],
                                  inputs=[model_data, metrics_data],
                                  compute_target=aml_compute,
                                  runconfig=run_automl_config,
                                  allow_reuse=False)

fairlearn_detect_fairness_step = PythonScriptStep(script_name="detect_fairness.py",
                                  source_directory=project_folder,
                                  name="Detect Fairness",
                                  arguments=["--fitted_model_name", ppb.get_pipeline_parameter_obj('fitted_model_name_fairlearn_detect_fairness_step'),
                                             "--model_data", ppb.get_pipeline_parameter_obj('model_data_fairlearn_detect_fairness_step'),
                                             "--dataset_name", ppb.get_pipeline_parameter_obj('dataset_name_fairlearn_detect_fairness_step'),
                                             "--output_fairness_dict", detect_fairness_step_output],
                                  compute_target=aml_compute,
                                  outputs=[detect_fairness_step_output],
                                  runconfig=run_fairlearn_detect_fairness_config,
                                  allow_reuse=False)

fairlearn_create_multimodels_fairness_step = PythonScriptStep(script_name="build_fairlearn_multi_models.py",
                                  source_directory=project_folder,
                                  name="Build Fairlearn Multi Model View",
                                  arguments=["--dataset_name", ppb.get_pipeline_parameter_obj('dataset_name_fairlearn_create_multimodels_fairness_step')],
                                  compute_target=aml_compute,
                                  runconfig=run_fairlearn_create_multimodel_view_config,
                                  allow_reuse=False)

fairlearn_upload_fairness_step = PythonScriptStep(script_name="upload_automl_fairness.py",
                                  source_directory=project_folder,
                                  name="Upload AutoML Model Fairness",
                                  arguments=["--dataset_name", ppb.get_pipeline_parameter_obj('dataset_name_fairlearn_detect_fairness_step'),
                                             "--output_fairness_dict", detect_fairness_step_output],
                                  compute_target=aml_compute,
                                  inputs=[detect_fairness_step_output],
                                  runconfig=run_fairlearn_create_multimodel_view_config,
                                  allow_reuse=False)

register_explainer_model_step = PythonScriptStep(script_name="register_explainer.py",
                                  source_directory=project_folder,
                                  name="Register Explainer Model",
                                  arguments=["--explainer_model_name", ppb.get_pipeline_parameter_obj('explainer_model_name_explainer_step'),
                                             "--fitted_model_name", ppb.get_pipeline_parameter_obj('fitted_model_name_explainer_step'),
                                             "--model_data", ppb.get_pipeline_parameter_obj('model_data_explainer_step'),
                                             "--dataset_name", ppb.get_pipeline_parameter_obj('dataset_name_explainer_step')],
                                  compute_target=aml_compute,
                                  runconfig=run_explainer_model_config,
                                  allow_reuse=False)

deploy_model_step = PythonScriptStep(script_name="deploy.py",
                                  source_directory=deploy_folder,
                                  name="Deploy Model",
                                  arguments=["--update_deployment", ppb.get_pipeline_parameter_obj('update_deployment_deploy_step'),
                                             "--dataset_name", ppb.get_pipeline_parameter_obj('dataset_name_deploy_step'),
                                             "--model_name", ppb.get_pipeline_parameter_obj('model_name_deploy_step'),
                                             "--service_name", ppb.get_pipeline_parameter_obj('service_name_deploy_step'),
                                             "--explainer_model_name", ppb.get_pipeline_parameter_obj('explainer_model_name_deploy_step'),
                                            ],
                                  compute_target=aml_compute,
                                  runconfig=run_automl_config,
                                  allow_reuse=False)

preprocessing_step.run_after(differential_privacy_step)
fairlearn_detect_fairness_step.run_after(register_model_step)
fairlearn_create_multimodels_fairness_step.run_after(fairlearn_upload_fairness_step)
register_explainer_model_step.run_after(register_model_step)
deploy_model_step.run_after(register_explainer_model_step)
steps = [differential_privacy_step, preprocessing_step, automl_step,
         fairlearn_detect_fairness_step, fairlearn_create_multimodels_fairness_step,
         fairlearn_upload_fairness_step, register_model_step,
         register_explainer_model_step, deploy_model_step]
print("Steps created")

## Pipeline Important Flags

### Update Deployment

**Note:** In the **Deploy Step** we have so many parameters to execute it. The *update_deployment* determines whether the step should create a ACI(Azure Container Instance). 

**True=Update Service**

**False=Generate deploy from scratch**

1. If *update_deployment* is set to *False*; the default is set to *False* because we typically do not want to update an ACI service cause we dont have anything created yet.

2. If *update_deployment* is set to *True*, a new run will always update an ACI Service that was created before.

You can change pipeline parameters on the following path: **notebooks/utils/params_config/pipeline_parameters**

### Retrain Status

**Note:** In the **Differential Privacy** and **Exploratory Analysis and Preprocessing** we have a retrain status Flag.

**True=Complete this steps, cause we dont need to do it on retrain process.**

**False=Generate differential privacy and Exploratory Analysis from scratch**

1. If *retrain_status* is set to *False*; the default is set to *False* because we typically want in the first execution create and analyze our training dataset. 

2. If *retrain_status* is set to *True*, a new run will always pass the Differential Pirvacy and Exploratory Analsysis execution. Just permit to create the new dataset for retrainin process on Exploratory Analysis and Preprocessing step

You can change pipeline parameters on the following path: **notebooks/utils/params_config/pipeline_parameters**

### Use Datadrift

**Note:** In the **Exploratory Analysis and Preprocessing** we have a use datadrift Flag.

**True = Split Dataset to have a Drift on our dataset by Sex**

**False = Do standard training dataset**

1. If *use_datadrift* is set to *True*; the default is set to *True* because we typically want in the first execution create a dataset with drift, in that case split the dataset by Sex (male/female). With that split, we train with males and do inferences when the model is on a ACI service. This females inferences will make that DataDrift detect a significant drift of the dataset an we can get results really simply. This split is only to demostrate Data Drift Detector execution but it is not relevant to do it if we wanna to do a good model training.

2. If *use_datadrift* is set to *False*, a new run will always make a standard training dataset.

You can change pipeline parameters on the following path: **notebooks/utils/params_config/pipeline_parameters**

### Allow reuse

**Note:** The flag *allow_reuse* determines whether the step should reuse previous results when run with the same settings/inputs. This flag's default value is *True*; the default is set to *True* because, when inputs and parameters have not changed, we typically do not want to re-run a given pipeline step. 

If *allow_reuse* is set to *False*, a new run will always be generated for this step during pipeline execution. The *allow_reuse* flag can come in handy in situations where you do *not* want to re-run a pipeline step.

## Build the pipeline
Once we have the steps (or steps collection), we can build the [pipeline](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipeline.pipeline?view=azure-ml-py). By deafult, all these steps will run in **parallel** once we submit the pipeline for run.

A pipeline is created with a list of steps and a workspace. Submit a pipeline using [submit](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment%28class%29?view=azure-ml-py#submit). When submit is called, a [PipelineRun](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipelinerun?view=azure-ml-py) is created which in turn creates [StepRun](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.steprun?view=azure-ml-py) objects for each step in the workflow.

In [ ]:
pipeline = Pipeline(workspace=ws, steps=steps)
print ("Pipeline is built")

## Validate the pipeline
You have the option to [validate](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipeline.pipeline?view=azure-ml-py#validate) the pipeline prior to submitting for run. The platform runs validation steps such as checking for circular dependencies and parameter checks etc. even if you do not explicitly call validate method.

In [ ]:
pipeline.validate()
print("Pipeline validation complete")

## Publish the pipeline to use via REST
Once you are satisfied with the results of your experiment, you may want to publish the pipeline to get a REST endpoint so the pipeline can be invoked later.

In [ ]:
published_training_pipeline = pipeline.publish(name="heart_disease_pipeline",
                                            description="This pipeline train a model to detect heart disease")
print("The published pipeline ID is {}".format(published_training_pipeline.id))

In [ ]:
pipeline_endpoint = PipelineEndpoint.publish(workspace=ws,
                                            name="responsible_ai_v1",
                                            pipeline=published_training_pipeline,
                                            description="This http pipeline train a model to detect heart disease")